<a href="https://colab.research.google.com/github/audreyemmely/pln/blob/main/pln_lista2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Resolução Lista 2**

---

Audrey Emmely Rodrigues Vasconcelos

Karen Nayara Gomes da Silva

###### Questão 1
Determine a distribuição de comprimentos dos textos (em quantidade de caracteres), listando estas quantidades e plotando um histograma. 

###### Questão 2
Aplique os seguintes passos de pré-processamento aos textos: 
* Remova todas as palavras que contêm números; 
* Converta as palavras para minúsculas; 
* Remova pontuação; 
* Tokenize os textos em palavras, gerando um dicionário único com *n* tokens e convertendo cada texto em um vetor de dimensão *n* com a respectiva contagem de palavras. 

Em seguida, encontre as palavras mais frequentes da base de textos. 


###### Questão 3
Aplique os seguintes passos de pré-processamento aos textos processados na questão anterior:

* Remova stopwords;
* Realize rotulação de POS;
* Realize stemização;

Exiba os resultados em alguns textos. Verifique quais são as palavras mais
frequentes e compare com as palavras mais frequentes da questão anterior.